In [1]:
import wisardpkg as wp
import numpy as np
import pandas as pd
from ucimlrepo import fetch_ucirepo 
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt



### Classe 1 : Has breast cancer
Se (idade $\notin [30, 50]$) e (menopausa = gl40) e (tamanho-tumor $\in [4, 19]$ OU tamanho-tumor $\in [29, 34]$ OU tamanho-tumor $\in [44, 49]$) e (inv-nodos $\notin [6, 9]$ e inv-nodos $\notin [12, 24]$) e (grau-maligno $\ne 3$) e (quad-mama $\ne$ direita-acima) e (irradiado = sim) Então Classe 0.

Se (idade $\notin [70]$) e (menopausa = premeno) e (tamanho-tumor $\in [0, 4]$ OU tamanho-tumor $\in [9, 14]$ OU tamanho-tumor $\in [19, 24]$ OU tamanho-tumor $\in [39, 59]$) e (inv-nodos $\notin [9, 12]$ e inv-nodos $\notin [24, 26]$) e (nodos-caps = 0) e (grau-maligno $\ne 3$) e (mama = direita) e (quad-mama $\ne$ central) e (irradiado = sim) Então Classe 0.

Se (idade $\notin [70]$) e (menopausa $\ne$ lt40) e (tamanho-tumor $\in [4, 9]$ OU tamanho-tumor $\in [14, 39]$ OU tamanho-tumor $\in [44, 59]$) e (inv-nodos $\notin [3, 15]$) e (nodos-caps = 0) e (grau-maligno = 1) e (mama = esquerda) e (quad-mama $\notin$ esquerda-acima e direita-acima) e (irradiado = sim) Então Classe 0.

Se (idade $\notin [40, 50]$) e (idade $\in [60, 70]$) e (menopausa = gl40) e (tamanho-tumor $\in [0, 9]$ OU tamanho-tumor $\in [14, 19]$ OU tamanho-tumor $\in [24, 29]$ OU tamanho-tumor $\in [34, 44]$) e (inv-nodos $\notin [0, 6]$ OU inv-nodos $\in [9, 15]$) e (quad-mama $\notin$ esquerda-baixo OU central) e (irradiado = sim) Então Classe 0.

Se (idade $\notin [40, 50]$ OU idade $\in [70, 80]$) e (menopausa = premeno) e (tamanho-tumor $\notin [4, 19]$) e (inv-nodos $\in [0, 3]$ OU inv-nodos $\in [9, 12]$) e (nodos-caps = 0) e (grau-maligno = 2) e (quad-mama $\ne$ esquerda-baixo) Então Classe 0.

In [3]:
def encode_breast_cancer(df, lista_pesos=[1,1,1,1,1,1,1,1,1]):
    # Age
    age_bins = ['20-29', '30-39', '40-49', '50-59', '60-69', '70-79']  # 6 bins
    def encode_age(age):
        return [int(age == b) for b in age_bins]

    # Menopause
    menopause_cats = ['lt40', 'ge40', 'premeno']  # 3 bits
    def encode_menopause(val):
        return [int(val == cat) for cat in menopause_cats]

    # Tumor-size
    tumor_bins = ['0-4', '9-May', '14-Oct', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-59']  # 11 bins
    def encode_tumor_size(size):
        return [int(size == b) for b in tumor_bins]

    # Inv-nodes
    inv_nodes_bins = ['0-2', '5-Mar', '8-Jun', '11-Sep', '14-Dec', '15-17', '24-26']  # 7 bins
    def encode_inv_nodes(nodes):
        return [int(nodes == b) for b in inv_nodes_bins]

    # Node-caps
    def encode_node_caps(val):
        return [int(val == 'no'), int(val == 'yes')]  # 2 bits

    # Deg-malig
    def encode_deg_malig(val):
        val = int(val)
        return [int(val == 1), int(val == 2), int(val == 3)]  # 3 bits

    # Breast
    def encode_breast(val):
        return [int(val == 'left'), int(val == 'right')]  # 2 bits

    # Breast-quad
    breast_quad_cats = ['left_up', 'left_low', 'right_up', 'right_low', 'central']  # 5 bits
    def encode_breast_quad(val):
        return [int(val == cat) for cat in breast_quad_cats]

    # Irradiat
    def encode_irradiat(val):
        return [int(val == 'yes'), int(val == 'no')]  # 2 bits

    if lista_pesos is None:
        lista_pesos = [1] * 9
    if len(lista_pesos) != 9:
        raise ValueError("lista_pesos deve ter exatamente 9 números inteiros.")

    encoded_rows = []
    for _, row in df.iterrows():
        encoded = []
        encoded += encode_age(row['age']) * lista_pesos[0]
        encoded += encode_menopause(row['menopause']) * lista_pesos[1]
        encoded += encode_tumor_size(row['tumor-size']) * lista_pesos[2]
        encoded += encode_inv_nodes(row['inv-nodes']) * lista_pesos[3]
        encoded += encode_node_caps(row['node-caps']) * lista_pesos[4]
        encoded += encode_deg_malig(row['deg-malig']) * lista_pesos[5]
        encoded += encode_breast(row['breast']) * lista_pesos[6]
        encoded += encode_breast_quad(row['breast-quad']) * lista_pesos[7]
        encoded += encode_irradiat(row['irradiat']) * lista_pesos[8]
        encoded_rows.append(encoded)
    return np.array(encoded_rows)

In [4]:
def thermometer_encode_breast_cancer(df, lista_pesos=[1,1,1,1,1,1,1,1,1]):
    # Age
    age_bins = ['20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
    def encode_age(age):
        idx = age_bins.index(age) if age in age_bins else -1
        return [1 if i <= idx else 0 for i in range(len(age_bins))]

    # Menopause
    menopause_cats = ['lt40', 'ge40', 'premeno']
    def encode_menopause(val):
        idx = menopause_cats.index(val) if val in menopause_cats else -1
        return [1 if i <= idx else 0 for i in range(len(menopause_cats))]

    # Tumor-size
    tumor_bins = ['0-4', '9-May', '14-Oct', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-59']
    def encode_tumor_size(size):
        idx = tumor_bins.index(size) if size in tumor_bins else -1
        return [1 if i <= idx else 0 for i in range(len(tumor_bins))]

    # Inv-nodes
    inv_nodes_bins = ['0-2', '5-Mar', '8-Jun', '11-Sep', '14-Dec', '15-17', '24-26']
    def encode_inv_nodes(nodes):
        idx = inv_nodes_bins.index(nodes) if nodes in inv_nodes_bins else -1
        return [1 if i <= idx else 0 for i in range(len(inv_nodes_bins))]

    # Node-caps
    def encode_node_caps(val):
        idx = 0 if val == 'no' else 1 if val == 'yes' else -1
        return [1 if i <= idx else 0 for i in range(2)]

    # Deg-malig
    def encode_deg_malig(val):
        val = int(val)
        idx = val - 1 if 1 <= val <= 3 else -1
        return [1 if i <= idx else 0 for i in range(3)]

    # Breast
    def encode_breast(val):
        idx = 0 if val == 'left' else 1 if val == 'right' else -1
        return [1 if i <= idx else 0 for i in range(2)]

    # Breast-quad
    breast_quad_cats = ['left_up', 'left_low', 'right_up', 'right_low', 'central']
    def encode_breast_quad(val):
        idx = breast_quad_cats.index(val) if val in breast_quad_cats else -1
        return [1 if i <= idx else 0 for i in range(len(breast_quad_cats))]

    # Irradiat
    def encode_irradiat(val):
        idx = 0 if val == 'yes' else 1 if val == 'no' else -1
        return [1 if i <= idx else 0 for i in range(2)]

    encoded_rows = []
    for _, row in df.iterrows():
        encoded = []
        encoded += encode_age(row['age']) * lista_pesos[0]
        encoded += encode_menopause(row['menopause']) * lista_pesos[1]
        encoded += encode_tumor_size(row['tumor-size']) * lista_pesos[2]
        encoded += encode_inv_nodes(row['inv-nodes']) * lista_pesos[3]
        encoded += encode_node_caps(row['node-caps']) * lista_pesos[4]
        encoded += encode_deg_malig(row['deg-malig']) * lista_pesos[5]
        encoded += encode_breast(row['breast']) * lista_pesos[6]
        encoded += encode_breast_quad(row['breast-quad']) * lista_pesos[7]
        encoded += encode_irradiat(row['irradiat']) * lista_pesos[8]
        encoded_rows.append(encoded)
    return np.array(encoded_rows)

# Data

In [5]:
breast_cancer = fetch_ucirepo(id=14) 

data = breast_cancer.data.original
data = data.dropna(subset=['breast-quad', 'node-caps'])

X = data[breast_cancer.data.features.columns]
y = data[breast_cancer.data.targets.columns]

## Misto 

In [23]:
parte_termometro = thermometer_encode_breast_cancer(X, lista_pesos=[1,0,1,1,0,0,0,0,0])
parte_onehot = encode_breast_cancer(X, lista_pesos=[0,1,0,0,1,1,1,1,1])
parte_mista = np.concatenate([parte_termometro, parte_onehot], axis=1)

def encode_y(y):
    return y.replace({'no-recurrence-events': 0, 'recurrence-events': 1})

encoded_y = encode_y(y)

X_mista_train, X_mista_test, y_mista_train, y_mista_test = train_test_split(parte_mista, encoded_y, test_size=0.2, random_state=42)

C:\Users\yasmin.romeiro\AppData\Local\Temp\ipykernel_24416\3085196323.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return y.replace({'no-recurrence-events': 0, 'recurrence-events': 1})


## Termometro

In [14]:
# Exemplo de uso:
encoded_X_balanceado = thermometer_encode_breast_cancer(X, lista_pesos=[2,4,1,2,6,4,6,2,6])
encoded_X = thermometer_encode_breast_cancer(X)
encoded_X_peso_distribuido = thermometer_encode_breast_cancer(X, lista_pesos= [1,1,1,6,12,12,1,1,12])

def encode_y(y):
    return y.replace({'no-recurrence-events': 0, 'recurrence-events': 1})

encoded_y = encode_y(y)

X_term_train, X_term_test, y_term_train, y_term_test = train_test_split(encoded_X, encoded_y, test_size=0.2, random_state=42)
X_term_bal_train, X_term_bal_test, y_term_bal_train, y_term_bal_test = train_test_split(encoded_X_balanceado, encoded_y, test_size=0.2, random_state=42)
X_term_corr_dif_all_train, X_term_corr_dif_all_test, y_term_corr_dif_all_train, y_term_corr_dif_all_test = train_test_split(encoded_X_peso_distribuido, encoded_y, test_size=0.2, random_state=42)


C:\Users\yasmin.romeiro\AppData\Local\Temp\ipykernel_17696\1425512593.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return y.replace({'no-recurrence-events': 0, 'recurrence-events': 1})


## One-hot

In [6]:
encoded_X = encode_breast_cancer(X)
print(len(encoded_X[0]))  

41


In [8]:
def encode_y(y):
    return y.replace({'no-recurrence-events': 0, 'recurrence-events': 1})

encoded_y = encode_y(y)

C:\Users\yasmin.romeiro\AppData\Local\Temp\ipykernel_1984\2485239602.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return y.replace({'no-recurrence-events': 0, 'recurrence-events': 1})


In [9]:
X_1hot_train, X_1hot_test, y_1hot_train, y_1hot_test = train_test_split(encoded_X, encoded_y, test_size=0.2, random_state=42)

## One-hot Colunas balanceadas

In [ ]:
# Exemplo de uso:
encoded_X = encode_breast_cancer(X, lista_pesos=[2,4,1,2,6,4,6,2,6])
print(len(encoded_X[0]))  

107


: 

In [19]:
def encode_y(y):
    return y.replace({'no-recurrence-events': 0, 'recurrence-events': 1})

encoded_y = encode_y(y)

C:\Users\yasmin.romeiro\AppData\Local\Temp\ipykernel_17696\2485239602.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return y.replace({'no-recurrence-events': 0, 'recurrence-events': 1})


In [20]:
from sklearn.model_selection import train_test_split
X_1hot_bal_train, X_1hot_bal_test, y_1hot_bal_train, y_1hot_bal_test = train_test_split(encoded_X, encoded_y, test_size=0.2, random_state=42)

## One-hot correlação das colunas

In [24]:
data_corr= data.rename(columns={'Class': 'target'})
data_corr = data_corr.astype(str)

In [25]:
from scipy.stats import chi2_contingency
import pandas as pd

results = []
for col in data_corr.columns.drop("target"):
    contingency = pd.crosstab(data_corr[col], data_corr["target"])
    chi2, p, dof, ex = chi2_contingency(contingency)
    results.append((col, chi2, p))

chi2_results_target = pd.DataFrame(results, columns=["Feature", "Chi2", "p-value"]).sort_values("p-value")

In [26]:
for results in chi2_results_target.itertuples():
    if results._3 < 0.05:
        print(f"Feature: {results.Feature}")

Feature: deg-malig
Feature: node-caps
Feature: inv-nodes
Feature: irradiat
Feature: tumor-size


### Não balanceadas

In [28]:
encoded_X = encode_breast_cancer(X, lista_pesos= [0,0,1,1,1,1,0,0,1])

def encode_y(y):
    return y.replace({'no-recurrence-events': 0, 'recurrence-events': 1})

encoded_y = encode_y(y)

C:\Users\yasmin.romeiro\AppData\Local\Temp\ipykernel_17696\948225654.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return y.replace({'no-recurrence-events': 0, 'recurrence-events': 1})


In [29]:
X_1hot_corr_train, X_1hot_corr_test, y_1hot_corr_train, y_1hot_corr_test = train_test_split(encoded_X, encoded_y, test_size=0.2, random_state=42)

### Balanceadas

In [30]:
encoded_X = encode_breast_cancer(X, lista_pesos= [0,0,1,2,6,4,0,0,6])

def encode_y(y):
    return y.replace({'no-recurrence-events': 0, 'recurrence-events': 1})

encoded_y = encode_y(y)

C:\Users\yasmin.romeiro\AppData\Local\Temp\ipykernel_17696\4100790109.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return y.replace({'no-recurrence-events': 0, 'recurrence-events': 1})


In [31]:
X_1hot_bal_corr_train, X_1hot_bal_corr_test, y_1hot_bal_corr_train, y_1hot_bal_corr_test = train_test_split(encoded_X, encoded_y, test_size=0.2, random_state=42)

### Diferentes pesos

In [32]:
encoded_X = encode_breast_cancer(X, lista_pesos= [0,0,1,6,12,12,0,0,12])

def encode_y(y):
    return y.replace({'no-recurrence-events': 0, 'recurrence-events': 1})

encoded_y = encode_y(y)

C:\Users\yasmin.romeiro\AppData\Local\Temp\ipykernel_17696\1946338939.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return y.replace({'no-recurrence-events': 0, 'recurrence-events': 1})


In [33]:
X_1hot_bal_corr_dif_train, X_1hot_bal_corr_dif_test, y_1hot_bal_corr_dif_train, y_1hot_bal_corr_dif_test = train_test_split(encoded_X, encoded_y, test_size=0.2, random_state=42)

### Diferentes Pesos + outras colunas

In [34]:
encoded_X = encode_breast_cancer(X, lista_pesos= [1,1,1,6,12,12,1,1,12])

def encode_y(y):
    return y.replace({'no-recurrence-events': 0, 'recurrence-events': 1})

encoded_y = encode_y(y)

C:\Users\yasmin.romeiro\AppData\Local\Temp\ipykernel_17696\3977274932.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return y.replace({'no-recurrence-events': 0, 'recurrence-events': 1})


In [35]:
X_1hot_corr_dif_all_train, X_1hot_corr_dif_all_test, y_1hot_corr_dif_all_train, y_1hot_corr_dif_all_test = train_test_split(encoded_X, encoded_y, test_size=0.2, random_state=42)

# Treinamento

## Mista

In [ ]:
def accuracy(y_pred, y_target):
  return np.where(y_pred == y_target, 1, 0).sum()/len(y_target)

In [ ]:
for addressSize in range(2, 20):
    for i in range(0,100):
        X = X_mista_train
        y = y_mista_train['Class'].values.astype(str)
        resultados = []
        wsd = wp.Wisard(addressSize, bleachingActivated= True, ignoreZero=False, verbose=True)
        time.sleep(3)
        wsd.train(X, y)
        out = wsd.classify(X_mista_test)
        acc = accuracy_score(y_mista_test['Class'].values.astype(str), out)
        resultados.append({'modelo': 'WiSARD', 'iteracao': i, 'tuple_size': addressSize, 'alfa': '-', 'acuracia': acc})
        pd.DataFrame(resultados).to_csv('resultados_BC_wisardpkd_mista.csv', mode='a', header=False, index=False)

In [65]:
ds = wp.DataSet()
for x, y in zip(X_mista_train, y_mista_train['Class'].values.astype(str)):
    ds.add(x.tolist(), y)

ds_test = wp.DataSet()
for x in X_mista_test:
    ds_test.add(x.tolist())

for addressSize in range(2, 20):
    for i in range(0,100):
        time.sleep(1)
        resultados = []
        model = wp.Wisard(addressSize, bleachingActivated= True, ignoreZero=False, verbose=True)
        model.train(ds) 
        out = model.classify(ds_test)
        acc = accuracy(out, y_mista_test['Class'].values.astype(str))
        resultados.append({'modelo': 'WiSARD', 'iteracao': i, 'tuple_size': addressSize, 'alfa': '-', 'acuracia': acc})
        pd.DataFrame(resultados).to_csv('resultados_BC_wisardpkd_mista_v2.csv', mode='a', header=False, index=False)


In [61]:
ds = wp.DataSet()
for x, y in zip(X_mista_train, y_mista_train['Class'].values.astype(str)):
    ds.add(x.tolist(), y)
model = wp.Wisard(5)
model.train(ds)

In [62]:
ds_test = wp.DataSet()
for x in X_mista_test:
    ds_test.add(x.tolist())
result = model.classify(ds_test)

In [63]:
def accuracy(y_pred, y_target):
  return np.where(y_pred == y_target, 1, 0).sum()/len(y_target)

accuracy(result, y_mista_test['Class'].values.astype(str))

np.float64(0.6785714285714286)

## One-hot 

In [ ]:
for addressSize in range(2, 20):
    for i in range(0,100):
        X = X_1hot_train
        y = y_1hot_train['Class'].values.astype(str)
        resultados = []
        wsd = wp.Wisard(addressSize, bleachingActivated= True, ignoreZero=False, verbose=True)
        time.sleep(2)
        wsd.train(X, y)
        out = wsd.classify(X_1hot_test)
        acc = accuracy_score(y_1hot_test['Class'].values.astype(str), out)
        resultados.append({'modelo': 'WiSARD', 'iteracao': i, 'tuple_size': addressSize, 'alfa': '-', 'acuracia': acc})
        pd.DataFrame(resultados).to_csv('resultados_BC_wisardpkd_onehot_v2.csv', mode='a', header=False, index=False)

## One hot balanceado

In [37]:
for addressSize in range(12, 20):
    for i in range(0,100):
        time.sleep(3)
        X = X_1hot_bal_train
        y = y_1hot_bal_train['Class'].values.astype(str)
        resultados = []
        wsd = wp.Wisard(addressSize, bleachingActivated= True, ignoreZero=False, verbose=True)
        wsd.train(X, y)
        out = wsd.classify(X_1hot_bal_test)
        acc = accuracy_score(y_1hot_bal_test['Class'].values.astype(str), out)
        resultados.append({'modelo': 'WiSARD', 'iteracao': i, 'tuple_size': addressSize, 'alfa': '-', 'acuracia': acc})
        pd.DataFrame(resultados).to_csv('resultados_BC_wisardpkd_onehot_balanceado_v2.csv', mode='a', header=False, index=False)

## Correlação

### One-hot 

In [38]:
for addressSize in range(2, 20):
    for i in range(0,100):
        time.sleep(3)
        X = X_1hot_corr_train
        y = y_1hot_corr_train['Class'].values.astype(str)
        resultados = []
        wsd = wp.Wisard(addressSize, bleachingActivated= True, ignoreZero=False, verbose=True)
        wsd.train(X, y)
        out = wsd.classify(X_1hot_corr_test)
        acc = accuracy_score(y_1hot_corr_test['Class'].values.astype(str), out)
        resultados.append({'modelo': 'WiSARD', 'iteracao': i, 'tuple_size': addressSize, 'alfa': '-', 'acuracia': acc})
        pd.DataFrame(resultados).to_csv('resultados_BC_wisardpkd_correlation_onehot_v2.csv', mode='a', header=False, index=False)

### Balanceada

In [39]:
for addressSize in range(2, 20):
    for i in range(0, 100):
        time.sleep(3)
        X = X_1hot_bal_corr_train
        y = y_1hot_bal_corr_train['Class'].values.astype(str)
        resultados = []
        wsd = wp.Wisard(addressSize, bleachingActivated= True, ignoreZero=False, verbose=True)
        wsd.train(X, y)
        out = wsd.classify(X_1hot_bal_corr_test)
        acc = accuracy_score(y_1hot_bal_corr_test['Class'].values.astype(str), out)
        resultados.append({'modelo': 'WiSARD', 'iteracao': i, 'tuple_size': addressSize, 'alfa': '-', 'acuracia': acc})
        pd.DataFrame(resultados).to_csv('resultados_BC_wisardpkd_correlation_balanceado_onehot_v2.csv', mode='a', header=False, index=False)

### Pesos Diferentes

In [40]:
for addressSize in range(2, 20):
    for i in range(0,100):
        time.sleep(3)
        X = X_1hot_bal_corr_dif_train
        y = y_1hot_bal_corr_dif_train['Class'].values.astype(str)
        resultados = []
        wsd = wp.Wisard(addressSize, bleachingActivated= True, ignoreZero=False, verbose=True)
        wsd.train(X, y)
        out = wsd.classify(X_1hot_bal_corr_dif_test)
        acc = accuracy_score(y_1hot_bal_corr_dif_test['Class'].values.astype(str), out)
        resultados.append({'modelo': 'WiSARD', 'iteracao': i, 'tuple_size': addressSize, 'alfa': '-', 'acuracia': acc})
        pd.DataFrame(resultados).to_csv('resultados_BC_wisardpkd_correlation_diffPesos_onehot_v2.csv', mode='a', header=False, index=False)

### Pesos Direferentes + outras colunas

In [41]:
for addressSize in range(2, 20):
    for i in range(0,100):
        time.sleep(3)
        X = X_1hot_corr_dif_all_train
        y = y_1hot_corr_dif_all_train['Class'].values.astype(str)
        resultados = []
        wsd = wp.Wisard(addressSize, bleachingActivated= True, ignoreZero=False, verbose=True)
        wsd.train(X, y)
        out = wsd.classify(X_1hot_corr_dif_all_test)
        acc = accuracy_score(y_1hot_corr_dif_all_test['Class'].values.astype(str), out)
        resultados.append({'modelo': 'WiSARD', 'iteracao': i, 'tuple_size': addressSize, 'alfa': '-', 'acuracia': acc})
        pd.DataFrame(resultados).to_csv('resultados_BC_wisardpkd_correlation_diffPesos_2_onehot_v2.csv', mode='a', header=False, index=False)

## Temometro

In [43]:
for addressSize in range(17, 20):
    for i in range(0,100):
        time.sleep(3)
        X = X_term_train
        y = y_term_train['Class'].values.astype(str)
        resultados = []
        wsd = wp.Wisard(addressSize, bleachingActivated= True, ignoreZero=False, verbose=True)
        wsd.train(X, y)
        out = wsd.classify(X_term_test)
        acc = accuracy_score(y_term_test['Class'].values.astype(str), out)
        resultados.append({'modelo': 'WiSARD', 'iteracao': i, 'tuple_size': addressSize, 'alfa': '-', 'acuracia': acc})
        pd.DataFrame(resultados).to_csv('resultados_BC_wisardpkd_termometro_v2.csv', mode='a', header=False, index=False)

In [44]:
for addressSize in range(2, 20):
    for i in range(0,100):
        time.sleep(3)
        X = X_term_bal_train
        y = y_term_bal_train['Class'].values.astype(str)
        resultados = []
        wsd = wp.Wisard(addressSize, bleachingActivated= True, ignoreZero=False, verbose=True)
        wsd.train(X, y)
        out = wsd.classify(X_term_bal_test)
        acc = accuracy_score(y_term_bal_test['Class'].values.astype(str), out)
        resultados.append({'modelo': 'WiSARD', 'iteracao': i, 'tuple_size': addressSize, 'alfa': '-', 'acuracia': acc})
        pd.DataFrame(resultados).to_csv('resultados_BC_wisardpkd_termometro_balanceado_v2.csv', mode='a', header=False, index=False)

In [45]:
for addressSize in range(2, 20):
    for i in range(0,100):
        time.sleep(3)
        X = X_term_corr_dif_all_train
        y = y_term_corr_dif_all_train['Class'].values.astype(str)
        resultados = []
        wsd = wp.Wisard(addressSize, bleachingActivated= True,  ignoreZero=False, verbose=True)
        wsd.train(X, y)
        out = wsd.classify(X_term_corr_dif_all_test)
        acc = accuracy_score(y_term_corr_dif_all_test['Class'].values.astype(str), out)
        resultados.append({'modelo': 'WiSARD', 'iteracao': i, 'tuple_size': addressSize, 'alfa': '-', 'acuracia': acc})
        pd.DataFrame(resultados).to_csv('resultados_BC_wisardpkd_termometro_diff_pesos_v2.csv', mode='a', header=False, index=False)